### Prerequisites:
The benbiohelpers package, which can be found here: https://github.com/bmorledge-hampton19/benbiohelpers
To install, download the repository and navigate to the benbiohelpers/python directory in the terminal. Then, run the command `pip install ./`.

The bedtools package. This can be installed using apt in the terminal with the following command: `sudo apt install bedtools`

### Set up data directory, data set names, and external data paths

This code establishes names and locations for the relevant data sets. All paths are defined relative to the directory containing the notebook.

In [ ]:
import os
# Define the relative path to the data directory containing the relevant input narrow peak files.
dataDirectory = os.path.abspath(os.path.join("..","..","data", "okita", "main_seCLIP_analysis"))

# Each binding protein should have its own directory at {dataDirectory}/{binding_protein_name}. This directory should contain
# input narrow peak files named in the following style: {binding_protein_name}_{repetition}.narrowPeak.
# Define the relevant binding protein names and their associated repetitions below.
bindingProteins = ["RBP_A", "RBP_I", "RBP_J", "RBP_K"]
repsByProtein = {"RBP_A" : ["rep1", "rep2", "rep3"],
                 "RBP_I" : ["rep1", "rep2", "rep3_combined"],
                 "RBP_J" : ["rep1_combined", "rep2", "rep3_combined"],
                 "RBP_K" : ["rep1", "rep2", "rep3"]}

# The following lines define paths to external data that should be the same for every analysis.
annotatedGenesFilePath = os.path.abspath(os.path.join(dataDirectory,"..","__external_data","all.locus_brief_info.7.0_sorted.tsv"))
exonsFilePath = os.path.abspath(os.path.join(dataDirectory,"..","__external_data","Osativa7_exons.bed"))
locusSortedExonsFilePath = os.path.abspath(os.path.join(dataDirectory,"..","__external_data","Osativa7_exons_locus_sorted.bed"))
genomeFastaFilePath = os.path.abspath(os.path.join(dataDirectory,"..","__external_data","Osativa7.fa"))

### Annotate Peaks
This code adds information on what gene(s) each peak is associated and also filters out any insignificant peaks.

Note: Make sure all seCLIP .narrowPeak files have their headers removed and are sorted (first on chromosome, then on start and end position, numerically) before running this step! Sorting can be achieved with the following terminal command:  
`sort -k1,1 -k2,2n -k3,3n -s -o path/to/file path/to/file`

In [ ]:
from AnnotatePeaks import annotatePeaks
for bindingProtein in bindingProteins:
    for rep in repsByProtein[bindingProtein]:
        annotatePeaks([os.path.join(dataDirectory,bindingProtein,f"{bindingProtein}_{rep}.narrowPeak")],
                      annotatedGenesFilePath, exonsFilePath, genomeFastaFilePath, True)

### Combine adjacent peaks

This code combines adjacent peaks, including those separated by exon boundaries.

In [ ]:
from CombineAdjacentPeaks import combineAdjacentPeaks
fullAnnotationFilePaths = list()
for bindingProtein in bindingProteins:
    for rep in repsByProtein[bindingProtein]:
        fullAnnotationFilePaths.append(
            os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_{rep}_full_annotation.bed")
        )
combineAdjacentPeaks(fullAnnotationFilePaths, locusSortedExonsFilePath, 10, 1)

### Find common loci between repetitions

This code creates a table of genomic loci containing peaks and records the number or repetitions containing peaks in each loci.

In [ ]:
from FindCommonLoci import findCommonLoci
for bindingProtein in bindingProteins:
    fullAnnotationFilePaths = list()
    for rep in repsByProtein[bindingProtein]:
        fullAnnotationFilePaths.append(
            os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_{rep}_full_annotation.bed")
        )
    findCommonLoci(fullAnnotationFilePaths, os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_common_loci.tsv"))

### Pool common loci (optional)

This code can be modified to return loci across multiple binding proteins that have peaks in a minimum number of replicates.
E.g., which loci occur in at least 2 RBP_A replicates and 2 RBP_I replicates.

In [ ]:
from PoolCommonLoci import poolCommonLoci

RBP_A_CommonLociFilePath = os.path.join(dataDirectory, "RBP_A", "RBP_A_common_loci.tsv")
RBP_I_CommonLociFilePath = os.path.join(dataDirectory, "RBP_I", "RBP_I_common_loci.tsv")

poolCommonLoci([RBP_A_CommonLociFilePath, RBP_I_CommonLociFilePath],
               os.path.join(dataDirectory, "RBP_A_RBP_I_min_2_reps_pooled_common_loci_min_2_reps.tsv"),
               minimumReplicates = 2, omitSingleDataSetLoci = True)

### Use the common loci file to format read sequences for STREME

This code uses the common loci file and annotated peaks file to retrieve sequences for analysis in STREME (or other motif callers).
A variety of parameters are available for filtering/modifying sequences. See examples below.


In [ ]:
from FormatReadSequencesForSTREME import formatReadSequencesForSTREME
for bindingProtein in bindingProteins:
    # Retrieve annotated peaks file paths.
    fullAnnotationFilePaths = list()
    fullAnnotationCombinedFilePaths = list()
    for rep in repsByProtein[bindingProtein]:
        fullAnnotationFilePaths.append(
            os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_{rep}_full_annotation.bed")
        )
        fullAnnotationCombinedFilePaths.append(
            os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_{rep}_full_annotation_combined.bed")
        )

    # Only filter on common loci in at least 2 files.
    formatReadSequencesForSTREME(
        fullAnnotationFilePaths, os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_STREME_input_minimal_filtering.fa"),
        genomeFastaFilePath, os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_common_loci.tsv"), 2
    )

    # Filter on common loci in at least 2 files and sequences <= 50 base pairs. Also expand sequences under 20 base pairs.
    formatReadSequencesForSTREME(
        fullAnnotationFilePaths, os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_STREME_input_20-50bp.fa"),
        genomeFastaFilePath, os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_common_loci.tsv"), 2, 50, 20
    )

    # Filter on common loci in at least 2 files. Also expand sequences under 20 base pairs.
    formatReadSequencesForSTREME(
        fullAnnotationFilePaths, os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_STREME_input_20bp_min.fa"),
        genomeFastaFilePath, os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_common_loci.tsv"), 2,
        minSequenceLength=20
    )

    # Use combined peaks. Filter on common loci in at least 2 files.
    # Call peaks from the 3' end, expanded 25 bp in the 5' direction and 25 bp in the 3' direction.
    # (Parameters based on relative positions of known prolamine binding motifs)
    formatReadSequencesForSTREME(
        fullAnnotationCombinedFilePaths,
        os.path.join(dataDirectory, bindingProtein,f"{bindingProtein}_STREME_input_combined_peaks_based_on_known_motif.fa"),
        genomeFastaFilePath, os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_common_loci.tsv"), 2,
        callFromThreePrimeEnd = True, fivePrimeExtension = 25, threePrimeExtension = 25
    )

    # Use non-combined peaks. Filter on common loci in at least 2 files.
    # Call peaks from the 3' end, expanded 25 bp in the 5' direction and 25 bp in the 3' direction.
    # (Parameters based on relative positions of known prolamine binding motifs)
    formatReadSequencesForSTREME(
        fullAnnotationFilePaths,
        os.path.join(dataDirectory, bindingProtein,f"{bindingProtein}_STREME_input_separate_peaks_based_on_known_motif.fa"),
        genomeFastaFilePath, os.path.join(dataDirectory, bindingProtein, f"{bindingProtein}_common_loci.tsv"), 2,
        callFromThreePrimeEnd = True, fivePrimeExtension = 25, threePrimeExtension = 25
    )